<a href="https://colab.research.google.com/github/AliZaiN-157/Sentiment-Aanlysis-with-Bert-Neural-Network/blob/main/Sentiment_Analysis_with_BERT_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Install Few Dependencies

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
from bs4 import BeautifulSoup

### 2. Configure Model

In [ ]:
tokenizer =AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### 3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('it was good, also delivered before the time ', return_tensors='pt')

In [ ]:
result = model(tokens)
result.logits

tensor([[-2.3284, -1.3999,  0.8664,  1.7375,  0.9398]],
       grad_fn=<AddmmBackward>)

In [ ]:
int(torch.argmax(result.logits))+1

4

### 4. Test Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]


In [ ]:
reviews[0]

'Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.'

### 5. Load Reviews with score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df['review'].iloc[0]
df

,review
0,Still one of the favorite coffee shop in Sydne...
1,Good coffee and toasts. Straight up and down -...
2,Ricotta hot cakes! These were so yummy. I ate ...
3,Ron & Jo are on the go down under and Wow! We...
4,Great coffee and vibe. That's all you need. C...
5,Great coffee and vibe. That's all you need. C...
6,We came for brunch twice in our week-long visi...
7,This is my fave brunch café in and around Sydn...
8,Delicious. The waitress was hot. The burger wa...
9,Rainy day in Sydney. Borrowed an umbrella and ...


In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,review,sentiment
0,Still one of the favorite coffee shop in Sydne...,5
1,Good coffee and toasts. Straight up and down -...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,Ron & Jo are on the go down under and Wow! We...,5
4,Great coffee and vibe. That's all you need. C...,5
5,Great coffee and vibe. That's all you need. C...,4
6,We came for brunch twice in our week-long visi...,4
7,This is my fave brunch café in and around Sydn...,5
8,Delicious. The waitress was hot. The burger wa...,4
9,Rainy day in Sydney. Borrowed an umbrella and ...,5
